## From Clustering to SAT, given $(I, k, D, d)$

k, number of cluster, d, minimum distance between different cluster.

Variables: $i_c$, where $i \in I$ is the item, $c \in [1, k]$ cluster number.  
$i_c = 1 \iff $ item i is in cluster c.

- Item $i$ should be in some cluster, eg. $x_1 \lor x_2 \lor \dots \lor x_k, y_1 \lor \dots \lor y_k, z_1 \lor \dots \lor z_k, \dots$  
$\forall i \in I: \lor_{1\le c \le k} i_c$

- Item $i$ shouldn't be in two clusters, eg $\overline{x_1} \lor \overline{x_2}, \overline{x_1} \lor \overline{x_3}, \overline{x_2} \lor \overline{x_3}$

$\forall i \in I, c_1 < c_2: \overline{i_{c_1}} \lor \overline{i_{c_2}}$

- Items at distance < d should belong to the same cluster.

eg, distance between x,y is < d, $x_1 \implies y_1, x_2 \implies y_2$, $\overline{x_1} \lor y_1, \overline{x_2} \lor y_2$  

$\forall i, j \in I$ s.t. $D[i,j] < d, \forall c: \overline{i_c} \lor j_c$

- cluster must be non-empty

$\forall c: \lor_{i \in I} i_c$

In [ ]:
"""
Solve the Clustering problem by reducing it to the SAT problem and solving it using PySat.

Clustering:
Given n items and pairwise distances between them,
split the items, if possible, into k clusters
so that the distance between any two items from different clusters is at least d.
"""


from pysat.formula import CNF
from pysat.solvers import Solver 


def i_c(i, c, k):
    # the index of the variable indicating that item i is assigned to cluster c (out of k clusters)
    return i*k + c + 1


def get_c(ic, k):
    # the number of the cluster linked to the variable ic
    return ic % k


def clustering2sat(distances, k, d):
    """Reduce Clustering to SAT.
    Input:  A square distance matrix: distances[i][j] = distance[j][i] is the distance between items i and j;
            the number of clusters k;
            and the minimal distance d between items to be placed into different clusters.
    Output: A CNF formula f that is satisfiable if and only if the items can be split into k clusters so that the
            distance between any two items from different clusters is at least d.
    
    """
    n = len(distances)
    f = CNF()

    # We use variables i_c indicating that item i is assigned to cluster c from {0, 1, ..., k - 1}
    for i in range(n):

        # item must be assigned to some cluster 
        f.append([i_c(i, c, k) for c in range(k)])

        # item cannot be assigned to two clusters
        for c1 in range(k):
            for c2 in range(c1 + 1, k):
                f.append([-i_c(i, c1, k), -i_c(i, c2, k)])

        # two items at distance < d from each other must be in the same cluster
        for j in range(i + 1, n):
            if distances[i][j] < d:
                for c in range(k):
                    f.append([-i_c(i, c, k), i_c(j, c, k)])
  
    # clusters must be non-empty
    for c in range(k):
        f.append([i_c(i, c, k) for i in range(n)])

    return f


def assemble_solution(model, k):
    """Extract a clustering from a satisfying assignment of the formula obtained via reduction.
    
    Input:  model is None if the formula is unsatisfiable;
            otherwise, it is a vector specifying values of the variables in the satisfying assignment.
    Output: a vector of numbers in [0, k) specifying clusters for corresponding items.
    
    """
    solution = None
    if model:
        solution = []
        for ic in range(len(model)):
            if model[ic] > 0:
                solution.append(get_c(ic, k))
    return solution

In [ ]:
def cluster(distances, k, d):
    """Use a SAT solver to solve the Clustering problem"""
    
    print("Reducing to SAT...")
    f = clustering2sat(distances, k, d)
    print(f.clauses)

    print("Solving SAT...")
    with Solver(bootstrap_with=f.clauses) as s:
        s.solve()
        model = s.get_model()
    
    print("Assembling solution...")
    return assemble_solution(model, k)

In [ ]:
# just one item
distances = [[0]]
print(cluster(distances, 1, 0))

In [ ]:
# Four items on a line with cooridinates 0, 2, 4, and 7.
distances = [[0, 2, 4, 7],
             [2, 0, 2, 5],
             [4, 2, 0, 3],
             [7, 5, 3, 0]
            ]
print(cluster(distances, 2, 3))
print()
print(cluster(distances, 2, 4))
print()
print(cluster(distances, 3, 3))

In [ ]:
# A slightly bigger example
distances = [[0,  12, 11, 12, 1,  1,  5],
             [12, 0,  13, 1,  12, 14, 15],
             [11, 13, 0,  12, 16, 13, 14],
             [12, 1,  12, 0,  16, 15, 12],
             [1,  12, 16, 16, 0,  2,  1],
             [1,  14, 13, 15, 2,  0,  2],
             [5,  15, 14, 12, 1,  2,  0],
            ]
print(cluster(distances, 3, 9))

In [ ]:
# What if, instead of trying to maximize the distance between items from different clusters,
# we wanted to minimize the distance between items from the same cluster?

# More precisely, consider the following Min-Clustering problem:
# Given n items and pairwise distances between them,
# split the items, if possible, into k clusters
# so that the distance between any two items in the same cluster is at most d.

# What should be changed in the clustering2sat reduction? 

## From vertex coloring to SAT

>Given a graph G and a number k, you must decide if the vertices of G can be colored in at most k colors so that adjacent vertices are colored differently.

$i$ represent nodes, $c$ represent color.

- Items must be colored

$\forall i_c \in I, \lor_{1\le c\le k} i_c$

- Items can not be colored twice, eg $\overline{x_1} \lor \overline{x_2}, \overline{x_1} \lor \overline{x_3}, \overline{x_2} \lor \overline{x_3}$

$\forall i \in I, c_1 < c_2: \overline{i_{c_1}} \lor \overline{i_{c_2}}$

- Items that are connected must not have same color

if $i$, $j$ are connected, $\forall c: \overline{i_{c}} \lor \overline{j_{c}}$

In [ ]:
from pysat.formula import CNF
from pysat.solvers import Solver


# You may want to define some auxiliary functions here
# your code here
def i_c(i, c, k):
    # the index of the variable indicating that item i is assigned to cluster c (out of k clusters)
    return (i-1)*k + c + 1

def coloring2sat(g, k):
    """Reduce Graph Coloring to SAT.
    
    Input:  An undirected graph g and a number k.
            The vertices of g are {1, ..., n} for some n.
            Here, g[0] = n and, for i > 0, g[i] = (u, v) specifies an edge between vertices u and  v.
    Output: A CNF-fomula f that is satisfiable if and only if g can be properly colored using at most k colors.
    
    """
    f = CNF()
    # your code here
    n = g[0]

    # iterate over all vertices
    # We use variables i_c indicating that item i is colored with c from {0, 1, ..., k - 1}
    for i in range(1, n+1):

        # item must be colored
        # print([i_c(i, c, k) for c in range(k)])
        f.append([i_c(i, c, k) for c in range(k)])

        # item cannot be colored by two colors
        for c1 in range(k):
            for c2 in range(c1 + 1, k):
                # print('tem cannot be colored by two colorss',[-i_c(i, c1, k), -i_c(i, c2, k)])
                f.append([-i_c(i, c1, k), -i_c(i, c2, k)])

    # iterate through all edges
    for pair in g:
        if type(pair) is tuple:
            for c in range(k):
                # connected vertices can not have same color
                # print([-i_c(pair[0], c, k), -i_c(pair[1], c, k)])
                f.append([-i_c(pair[0], c, k), -i_c(pair[1], c, k)])
    
    return f


def assemble_solution(model, k):
    """Extract a k-coloring of a graph g from a satisfying assignment of the formula obtained from g via reduction.
    
    Input:  A model is None if the formula is unsatisfiable;
            otherwise it is a list specifying the values of the variables in the satisfying assignment:
            for each variable i, the list contains i if variable i is true and -i if variable i is false.
    Output: A list of numbers in [1, k] specifying a k-coloring of g:
            solution[0] = k, and, for v > 0, solution[v] is a number in [1, k], the color of vertex v. 
    
    """
    solution = None
    if model:
    # your code here
        solution = [k]
        for ic in range(len(model)):
            if model[ic] > 0:
                solution.append(ic % k + 1)
    print(solution)
    return solution

def color(g, k):
    """Use a SAT solver to solve the Graph Coloring problem"""
    
    f = coloring2sat(g, k)
    
    with Solver(bootstrap_with=f.clauses) as s:
        s.solve()
        model = s.get_model()
    
    print(model)
    return assemble_solution(model, k)


def check_coloring(g, k, coloring):
    """Check if coloring is a valid k-coloring of graph g."""
    
    # coloring must be a list specifying the total number of colors and a color for every vertex
    assert coloring is not None
    assert len(coloring) == g[0] + 1
    assert coloring[0] <= k
    
    # every color is a number in [1, k]
    for c in coloring:
        assert 0 < c <= coloring[0]

    # two vertices connected by an edge must have different colors
    for (u, v) in g[1:]:
        assert coloring[u] != coloring[v]
            
            
# g = [1]
# assert color(g, 1) == [1, 1]

g = [7,
     (1, 2),
     (2, 3),
     (2, 5),
     (2, 6),
     (3, 4),
     (3, 5),
     (3, 6),
     (3, 7),
     (4, 5),
     (4, 6),
     (4, 7)
    ]

g = [5,
     (1, 2),
     (1, 3),
     (1, 4),
     (2, 3),
     (3, 4),
     (4, 5),
    ]

# coloring2sat(g, 3)
# color(g, 3)

check_coloring(g, 3, color(g, 3))
